In [ ]:
!pip install mplfinance
!pip install yfinance

In [ ]:
from datetime import timedelta, date
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mf
import yfinance as yf

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Flatten, Dense, Dropout, LSTMCell
%matplotlib inline

from keras import optimizers
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
def format_data(data):

    #format date and labels
    new_format = "%Y-%m-%dT%H:%M:%SZ"
    data['datetime'] = pd.to_datetime(data['datetime'], format=new_format)
    data.rename(columns={'datetime':'Date'}, inplace=True)
    data.rename(columns={'open_price':'Open'}, inplace=True)
    data.rename(columns={'close_price':'Close'}, inplace=True)
    data.rename(columns={'high_price':'High'}, inplace=True)
    data.rename(columns={'low_price':'Low'}, inplace=True)
    data.rename(columns={'volume':'Volume'}, inplace=True)
    data.rename(columns={'symbol':'Symbol'}, inplace=True)
    #data.rename(columns={'symbol':'Label'}, inplace=True)
    #data.reset_index(drop=True, inplace=True)
    #data.set_index('Date', inplace = True)
    
    return data

In [ ]:
# original.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
data = pd.read_csv('/kaggle/input/5years-dailystock-quotes/original.csv', delimiter=',')
data.dataframeName = 'original.csv'
data = format_data(data)
nRow, nCol = data.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
path = '/kaggle/working/'
shutil.os.mkdir(path + "model")
shutil.os.mkdir(path + "predict")
shutil.os.mkdir(path + "Candle_Data")
shutil.os.mkdir(path + "Candle_Data/Up")
shutil.os.mkdir(path + "Candle_Data/Down")

In [ ]:
tick=['FBNC', 'BANF', 'SLCT', 'SCHW']

for company in tick:
    print(company)
    ticker = data['Symbol']==company
    df = data[ticker].loc[data.first_valid_index():]
    df.reset_index(drop=True, inplace=True)
    df_pricing=df.copy()
    #set date as index for plots
    df_pricing.set_index('Date', inplace = True)
    
    n_days = 5
    window = 180
    fraction_movement=0.037
    df['Trend']=None
    
    for i in range(len(df)):
        try :
            for n in range(n_days): 
                if  df.loc[i,'Close'] - df.loc[i+n,'Close'] >= fraction_movement*df.loc[i,'Close']:
                    df.loc[i,'Trend'] ='Down'
                    
                    if i > window :
                        mf.plot(df_pricing[i-window:i], 
                            type = 'candle',
                            style =  'yahoo',
                            volume = True,
                            show_nontrading = True,
                            axisoff=True,
                            mav = (2,5,10),
                            closefig=True,
                            savefig='/kaggle/working/Candle_Data/Down/{0}{1}.png'.format(df_pricing['Symbol'][i],i)
                               )
                        
                    #print('Down',i,n)
                    break
                    
                elif df.loc[i+n,'Close'] - df.loc[i,'Close'] >= fraction_movement*df.loc[i,'Close'] :
                    df.loc[i,'Trend']='Up'
                    
                    if i > window :    
                        mf.plot(df_pricing[i-window:i], 
                            type = 'candle',
                            style =  'yahoo',
                            volume = True,
                            show_nontrading = True,
                            axisoff=True,
                            mav = (2,5,10),
                            closefig=True,
                            savefig='/kaggle/working/Candle_Data/Up/{0}{1}.png'.format(df_pricing['Symbol'][i],i)
                               )

                    #print('Up',i,n)
                    break
                    
                else :
                    df.loc[i,'Trend']= 'No Trend'
        except :
            #print(i)
            pass

In [ ]:
# Input the size of your sample images
img_width, img_height = 478, 339
# Enter the number of samples, training + validation
#nb_train_samples = 13204
#nb_validation_samples = 1412
nb_filters1 = 16
nb_filters2 = 32
nb_filters3 = 64
conv1_size = 4
conv2_size = 2
conv3_size = 6
pool_size = 2
# We have 2 classes, up and down
classes_num = 2
batch_size = 128
lr = 0.002
chanDim =3

In [ ]:
model = tf.keras.Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, padding ='same', input_shape=(img_height, img_width , 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, padding ="same"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format="channels_last"))

model.add(Convolution2D(nb_filters3, conv3_size, conv3_size, padding ='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format="channels_last"))

model.add(Flatten())
model.add(Dense(1024))

model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(classes_num, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

In [ ]:
validation_split = 0.2
data_dir = '/kaggle/working/Candle_Data/'

train_datagen = ImageDataGenerator(
    validation_split=validation_split, 
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.02,
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    shuffle=True,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
"""
Tensorboard log
"""
target_dir = "/kaggle/working/model/weights.hdf5"
#if not os.path.exists(target_dir):
#  os.mkdir(target_dir)
#model.save('./src/models/model.h5')
#model.save_weights('./src/models/weights.h5')

checkpoint = ModelCheckpoint(target_dir, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
epochs = 10
model.fit_generator(
    train_generator,
    #steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    shuffle=True,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    #validation_steps=nb_validation_samples//batch_size
)

In [ ]:
#need to create heatmap

In [ ]:
#lets grab some data and predict!

today = date.today()
end = today.strftime("%Y-%m-%d")
print(end)
endDate = date.today() - timedelta(days=window +30)
start = endDate.strftime("%Y-%m-%d")
print(start)

In [ ]:
df_yahoo = yf.download('FBNC',
                        start=start,
                        end=end,
                        progress=False)

#grab last n_days days
df_yahoo = df_yahoo[-window:]
df_yahoo

In [ ]:
#take a look
mf.plot(df_yahoo, 
        type = 'candle',
        style =  'yahoo',
        volume = True,
        show_nontrading = True,
        axisoff=True,
        mav = (2,5,10)
       )

#save
mf.plot(df_yahoo, 
        type = 'candle',
        style =  'yahoo',
        volume = True,
        show_nontrading = True,
        axisoff=True,
        mav = (2,5,10),
        savefig='/kaggle/working/predict/tomorrows_prediction.png'
       )


In [ ]:
def predict():
    
    x = load_img('/kaggle/working/predict/tomorrows_prediction.png', target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    if result[0] > result[1]:
        if result[0] > 0.55:
            print("Predicted answer: Buy")
            answer = 'buy'
            print(result)
            print(array)
        else:
            print("Predicted answer: Not confident buy")
            answer = 'n/a'
            print(result)
        
    else:
        if result[1] > 0.55:
            print("Predicted answer: Sell")
            answer = 'sell'
            print(result)
        else:
            print("Predicted answer: Not confident sell")
            answer = 'n/a'
            print(result)

    return answer

In [ ]:
predict()